
# Assignment 3 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 3, PROBLEM 1
Maximum Points = 8


Download the updated data folder from the course github website or just download directly the file [https://github.com/datascience-intro/1MS041-2025/blob/main/notebooks/data/smhi.csv](https://github.com/datascience-intro/1MS041-2025/blob/main/notebooks/data/smhi.csv) from the github website and put it inside your data folder, i.e. you want the path `data/smhi.csv`. The data was aquired from SMHI (Swedish Meteorological and Hydrological Institute) and constitutes per hour measurements of wind in the Uppsala Aut station. The data consists of windspeed and direction. Your goal is to load the data and work with it a bit. The code you produce should load the file as it is, please do not alter the file as the autograder will only have access to the original file.

The file information is in Swedish so you need to use some translation service, for instance `Google translate` or ChatGPT.

1. [2p] Load the file, for instance using the `csv` package. Put the wind-direction as a numpy array and the wind-speed as another numpy array.
2. [2p] Use the wind-direction (see [Wikipedia](https://en.wikipedia.org/wiki/Wind_direction)) which is an angle in degrees and convert it into a point on the unit circle **which is the direction the wind is blowing to** (compare to definition of radians [Wikipedia](https://en.wikipedia.org/wiki/Radian)). Store the `x_coordinate` as one array and the `y_coordinate` as another. From these coordinates, construct the wind-velocity vector.
3. [2p] Calculate the average wind velocity and convert it back to direction and compare it to just taking average of the wind direction as given in the data-file.
4. [2p] The wind velocity is a $2$-dimensional random variable, calculate the empirical covariance matrix which should be a numpy array of shape (2,2).

For you to wonder about, is it more likely for you to have headwind or not when going to the university in the morning.

In [43]:
import csv

# Empty Python lists where we'll store the values
wind_direction = []
wind_speed = []

# Open the original Swedish CSV file
with open("data/smhi.csv", "r", encoding="utf-8-sig") as f:
    # Use ; as delimiter (that's how SMHI exports)
    reader = csv.reader(f, delimiter=";")
    header_found = False

    for row in reader:
        # Skip completely empty rows
        if not row:
            continue

        # Check when we have reached the data header row
        # First column is "Datum" in Swedish
        if row[0].strip() == "Datum":
            header_found = True
            continue

        # Skip everything until we have seen the header "Datum;Tid (UTC);..."
        if not header_found:
            continue

        # We expect at least 5 columns: Datum, Tid, Vindriktning, Kvalitet, Vindhastighet, ...
        if len(row) < 5:
            continue

        try:
            # Column 2: Vindriktning (degrees)
            wd = float(row[2].replace(",", "."))

            # Column 4: Vindhastighet (m/s)
            ws = float(row[4].replace(",", "."))
        except ValueError:
            # If conversion to float fails (e.g. text instead of number), skip the row
            continue

        # Store the numeric values
        wind_direction.append(wd)
        wind_speed.append(ws)

# Convert to NumPy arrays for further calculations
import numpy as np

problem1_wind_direction = np.array(wind_direction)
problem1_wind_speed = np.array(wind_speed)

print(problem1_wind_direction)
print(problem1_wind_speed)

[348.  43. 185. ... 269. 281. 252.]
[0.6 0.2 0.1 ... 1.6 1.4 1.4]


In [51]:

# The wind direction is given as a compass direction in degrees (0-360)
# convert it to x and y coordinates using the standard mathematical convention
theta = np.deg2rad(problem1_wind_direction)  # convert degrees to radians
problem1_wind_direction_x_coordinate = -np.sin(theta)
problem1_wind_direction_y_coordinate = -np.cos(theta)
print(problem1_wind_direction_x_coordinate)
print(problem1_wind_direction_y_coordinate)

problem1_wind_velocity_x_coordinate = wind_speed * problem1_wind_direction_x_coordinate
problem1_wind_velocity_y_coordinate = wind_speed * problem1_wind_direction_y_coordinate
#print(problem1_wind_velocity_x_coordinate)
#print(problem1_wind_velocity_y_coordinate)
wind_velocity = np.vstack((problem1_wind_velocity_x_coordinate, problem1_wind_velocity_y_coordinate)).T
print(wind_velocity)

[ 0.20791169 -0.68199836  0.08715574 ...  0.9998477   0.98162718
  0.95105652]
[-0.9781476  -0.7313537   0.9961947  ...  0.01745241 -0.190809
  0.30901699]
[[ 0.12474701 -0.58688856]
 [-0.13639967 -0.14627074]
 [ 0.00871557  0.09961947]
 ...
 [ 1.59975631  0.02792385]
 [ 1.37427806 -0.26713259]
 [ 1.33147912  0.43262379]]


In [61]:

# Put the average wind velocity x and y coordinates here in these variables
problem1_average_wind_velocity_x_coordinate = np.mean(problem1_wind_velocity_x_coordinate)
problem1_average_wind_velocity_y_coordinate = np.mean(problem1_wind_velocity_y_coordinate)
print(problem1_average_wind_velocity_x_coordinate)
print(problem1_average_wind_velocity_y_coordinate)
# First calculate the angle of the average wind velocity vector in degrees
alpha_vel = np.arctan2(problem1_average_wind_velocity_y_coordinate, problem1_average_wind_velocity_x_coordinate) 
problem1_average_wind_velocity_angle_degrees = np.rad2deg(alpha_vel) % 360
print(problem1_average_wind_velocity_angle_degrees)
# Then calculate the average angle of the wind direction in degrees (using the wind direction in the data)
# Mean of unit direction vectors (ignoring wind speed)
mean_dir_x = np.mean(problem1_wind_direction_x_coordinate)
mean_dir_y = np.mean(problem1_wind_direction_y_coordinate)
# Angle of this mean direction (math convention, TO)
alpha_dir = np.arctan2(mean_dir_y, mean_dir_x)
problem1_average_wind_direction_angle_degrees = np.rad2deg(alpha_dir) % 360
print(problem1_average_wind_direction_angle_degrees)
def same_angle():
    return problem1_average_wind_direction_angle_degrees == np.mean(wind_direction)
print(np.mean(wind_direction))
# Finally, are they the same? Answer as a boolean value (True or False)
problem1_same_angle = same_angle()
print(problem1_same_angle)

0.3807312196610262
1.038909498722117
69.87353002120085
70.6277046215604
192.281280627246
False


In [53]:
problem1_wind_velocity_covariance_matrix = np.cov(problem1_wind_velocity_x_coordinate, problem1_wind_velocity_y_coordinate, ddof=1)
print(problem1_wind_velocity_covariance_matrix)

[[1.89213988 0.06366291]
 [0.06366291 2.45331787]]


---
## Assignment 3, PROBLEM 2
Maximum Points = 8


For this problem you will need the [pandas](https://pandas.pydata.org/) package and the [sklearn](https://scikit-learn.org/stable/) package. Inside the `data` folder from the course website you will find a file called `indoor_train.csv`, this file includes a bunch of positions in (X,Y,Z) and also a location number. The idea is to assign a room number (Location) to the coordinates (X,Y,Z).

1. [2p] Take the data in the file `indoor_train.csv` and load it using pandas into a dataframe `df_train`
2. [3p] From this dataframe `df_train`, create two numpy arrays, one `Xtrain` and `Ytrain`, they should have sizes `(1154,3)` and `(1154,)` respectively. Their `dtype` should be `float64` and `int64` respectively.
3. [3p] Train a Support Vector Classifier, `sklearn.svc.SVC`, on `Xtrain, Ytrain` with `kernel='linear'` and name the trained model `svc_train`.

To mimic how [kaggle](https://www.kaggle.com/) works, the Autograder has access to a hidden test-set and will test your fitted model.

In [75]:
import pandas as pd
df_train = pd.read_csv("data/indoor_train.csv")
print(df_train)

      Position X   Position Y  Position Z  Location
0           32.0         15.0         4.4        18
1            8.0         17.0         1.5         9
2            4.0         13.0         4.4        13
3           39.0         16.0         4.4        18
4           34.0         12.0         7.6        15
...          ...          ...         ...       ...
1149        39.0          9.0         1.5        11
1150        38.0         26.0         4.4         6
1151         4.0          7.0         1.5         2
1152        21.0         19.0         1.5        17
1153        25.0         19.0         7.6        19

[1154 rows x 4 columns]


In [80]:
Xtrain = df_train[['Position X', ' Position Y', 'Position Z']].to_numpy(dtype="float64")
Ytrain = df_train["Location"].to_numpy(dtype="int64")
print(Xtrain)
print(Ytrain)

[[32.  15.   4.4]
 [ 8.  17.   1.5]
 [ 4.  13.   4.4]
 ...
 [ 4.   7.   1.5]
 [21.  19.   1.5]
 [25.  19.   7.6]]
[18  9 13 ...  2 17 19]


In [81]:
from sklearn.svm import SVC
model = SVC(kernel="linear")
svc_train = model.fit(Xtrain, Ytrain)

---
## Assignment 3, PROBLEM 3
Maximum Points = 8


Let us build a proportional model ($\mathbb{P}(Y=1 \mid X) = G(\beta_0+\beta \cdot X)$ where $G$ is the logistic function) for the spam vs not spam data. Here we assume that the features are presence vs not presence of a word, let $X_1,X_2,X_3$ denote the presence (1) or absence (0) of the words $("free", "prize", "win")$.

1. [2p] Load the file `data/spam.csv` and create two numpy arrays, `problem3_X` which has shape **(n_texts,3)** where each feature in `problem3_X` corresponds to $X_1,X_2,X_3$ from above, `problem3_Y` which has shape **(n_texts,)** and consists of a $1$ if the email is spam and $0$ if it is not. Split this data into a train-calibration-test sets where we have the split $40\%$, $20\%$, $40\%$, put this data in the designated variables in the code cell.

2. [2p] Follow the calculation from the lecture notes where we derive the logistic regression and implement the final loss function inside the class `ProportionalSpam`. You can use the `Test` cell to check that it gives the correct value for a test-point.

3. [2p] Train the model `problem3_ps` on the training data. The goal is to calibrate the probabilities output from the model. Start by creating a new variable `problem3_X_pred` (shape `(n_samples,1)`) which consists of the predictions of `problem3_ps` on the calibration dataset. Then train a calibration model using `sklearn.tree.DecisionTreeRegressor`, store this trained model in `problem3_calibrator`. Recall that calibration error is the following for a fixed function $f$
$$
    \sqrt{\mathbb{E}[|\mathbb{E}[Y \mid f(X)] - f(X)|^2]}.
$$

4. [2p] Use the trained model `problem3_ps` and the calibrator `problem3_calibrator` to make final predictions on the testing data, store the prediction in `problem3_final_predictions`. 

In [144]:
import pandas as pd
import numpy as np
df = pd.read_csv("data/spam.csv", encoding="ISO-8859-1", usecols=[0, 1])
df = df.dropna(axis=1, how="all")
X1 = df["v2"].str.contains(r"\bfree\b", case=False, na = False).astype(int).to_numpy()
X2 = df["v2"].str.contains(r"\bprize\b", case=False, na = False).astype(int).to_numpy()
X3 = df["v2"].str.contains(r"\bwin\b", case=False, na = False).astype(int).to_numpy()
problem3_X = np.column_stack((X1, X2, X3))
#print(problem3_X)
problem3_Y = (df["v1"] == "spam").astype(int).to_numpy()
#print(problem3_Y)
n = problem3_X.shape[0]
rng = np.random.default_rng(seed=0)
perm = rng.permutation(n)           # random permutation of indices 0,1,..., n-1
train_end = int(0.4 * n)
calib_end = int(0.6 * n)
train_idx = perm[:train_end]
calib_idx = perm[train_end:calib_end]
test_idx = perm[calib_end:]

problem3_X_train = problem3_X[train_idx]
problem3_X_calib = problem3_X[calib_idx]
problem3_X_test = problem3_X[test_idx]

problem3_Y_train = problem3_Y[train_idx]
problem3_Y_calib = problem3_Y[calib_idx]
problem3_Y_test = problem3_Y[test_idx]

print(problem3_X_train.shape,problem3_X_calib.shape,problem3_X_test.shape,problem3_Y_train.shape,problem3_Y_calib.shape,problem3_Y_test.shape)

(2228, 3) (1115, 3) (2229, 3) (2228,) (1115,) (2229,)


In [145]:


class ProportionalSpam(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        z = coeffs[0] + np.dot(X, coeffs[1:])                       # β0 + Χ * β
        G = 1 / (1 + np.exp(-z))                                      # logistic function
        G = np.clip(G, 1e-12, 1- 1e-12)
        return -np.mean(Y*np.log(G) + (1 -Y)*np.log(1 - G))

    def fit(self,X,Y):
        import numpy as np
        from scipy import optimize

        #Use the f above together with an optimization method from scipy
        #to find the coefficients of the model
        opt_loss = lambda coeffs: self.loss(X,Y,coeffs)
        initial_arguments = np.zeros(shape=X.shape[1]+1)
        self.result = optimize.minimize(opt_loss, initial_arguments,method='cg')
        self.coeffs = self.result.x

    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            G = lambda x: np.exp(x)/(1+np.exp(x))
            return np.round(10*G(np.dot(X,self.coeffs[1:])+self.coeffs[0]))/10 # This rounding is to help you with the calibration

In [146]:

problem3_ps = ProportionalSpam()
problem3_ps.fit(problem3_X_train, problem3_Y_train)
problem3_X_pred = problem3_ps.predict(problem3_X_calib).reshape(-1,1)        # raw uncalibrated probabilities from the logistic model on the calibration dataset

from sklearn.tree import DecisionTreeRegressor
problem3_calibrator = DecisionTreeRegressor(
    max_depth=3,        # small depth to avoid overfitting                
    random_state=0      # reproducibility
)
problem3_calibrator.fit(problem3_X_pred, problem3_Y_calib)

,criterion,'squared_error'
,splitter,'best'
,max_depth,3
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [147]:
problem3_test_raw = problem3_ps.predict(problem3_X_test).reshape(-1, 1)     # raw probabilities on the test dataset
problem3_final_predictions = problem3_calibrator.predict(problem3_test_raw)
print(problem3_final_predictions)

[0.10268714 0.10268714 0.10268714 ... 0.10268714 0.10268714 0.10268714]


---
#### Local Test for Assignment 3, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [129]:
try:
    import numpy as np
    test_instance = ProportionalSpam()
    test_loss = test_instance.loss(np.array([[1,0,1],[0,1,1]]),np.array([1,0]),np.array([1.2,0.4,0.3,0.9]))
    assert (np.abs(test_loss-1.2828629432232497) < 1e-6)
    print("Your loss was correct for a test point")
except:
    print("Your loss was not correct on a test point")

Your loss was correct for a test point
